In [58]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, f1_score, accuracy_score
import torch.nn.functional as F 
import pandas as pd

In [2]:
MAX_LEN = 174
BATCH_SIZE = 64
EPOCHS = 5
MAX_GRAD_NORM = 5
MODEL_NAME = 'bert-base-uncased'
MODEL_PATH = 'ner_model_from_final1.pth'

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
def clean_tag(tag):
    # Ensure tags are in the correct format
    if tag.count('-') > 1:
        prefix, entity = tag.split('-', 1)
        tag = f"{prefix}-{entity.replace('-', '')}"
    return tag

In [5]:
def read_names(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        names = [name.strip().lower() for name in file.readlines()]
    return names

In [6]:
def read_data(file_path):
    sentences, labels = [], []
    sentence, label = [], []
    with open(file_path, encoding="utf-8") as file:
        for line in file:
            if line.startswith("#"):
                continue
            elif line == "\n":
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
            else:
                parts = line.strip().split("\t")
                sentence.append(parts[1].lower())  # Convert the token to lowercase before appending
                label.append(clean_tag(parts[2]))
    if sentence:
        sentences.append(sentence)
        labels.append(label)
    return sentences, labels

In [7]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

In [8]:
train_tokens, train_tags = read_data("./tagged_sentences_train.iob2")

In [9]:
test_tokens, test_tags = read_data("./tagged_sentences_test.iob2")

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
data = {'sentence': [" ".join(sentence) for sentence in train_tokens],
        'word_labels': [",".join(tags) for tags in train_tags]}

df = pd.DataFrame(data)
df.head()

,sentence,word_labels
0,this book is largely concerned with hobbits an...,"O,O,O,O,O,O,B-CHAR,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
1,further information will also be found in the ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,..."
2,that story was derived from the earlier chapte...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-CHAR,O,O,O,B-CHA..."
3,many however may wish to know more about this ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,for such readers a few notes on the more impor...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-CHAR,O,O,O,O,O..."


In [12]:
data_test = {'sentence': [" ".join(sentence) for sentence in test_tokens],
        'word_labels': [",".join(tags) for tags in test_tags]}

df_test = pd.DataFrame(data_test)
df_test.head()

,sentence,word_labels
0,oh smeagol ive got one ive got a fish smeagol ...,"O,B-CHAR,O,O,O,O,O,O,O,B-CHAR,I-CHAR,O,O,O,O,O..."
1,because it s my birthday and i wants it,"O,O,O,O,O,O,O,O,O"
2,my precious,"O,O"
3,they cursed us murderer murderer they called us,"O,O,O,O,O,O,O,O"
4,they cursed us and drove us away,"O,O,O,O,O,O,O"


In [13]:
# Split the word_labels into individual tags and get unique tags
all_tags = [tag for tags in df['word_labels'] for tag in tags.split(",")]
unique_tags = set(all_tags)


In [14]:
# Create mappings
labels_to_ids = {k: v for v, k in enumerate(unique_tags)}
ids_to_labels = {v: k for k, v in labels_to_ids.items()}

In [15]:

# Display the mappings
print("labels_to_ids:", labels_to_ids)
print("ids_to_labels:", ids_to_labels)

labels_to_ids: {'B-ORG': 0, 'B-CHAR': 1, 'I-CHAR': 2, 'I-LOC': 3, 'B-LOC': 4, 'O': 5}
ids_to_labels: {0: 'B-ORG', 1: 'B-CHAR', 2: 'I-CHAR', 3: 'I-LOC', 4: 'B-LOC', 5: 'O'}


In [16]:
# Split the word_labels into individual tags and get unique tags
all_tags_test = [tag for tags in df_test['word_labels'] for tag in tags.split(",")]
unique_tags_test = set(all_tags_test)

# Create mappings
labels_to_ids_test = {k: v for v, k in enumerate(unique_tags_test)}
ids_to_labels_test = {v: k for k, v in labels_to_ids.items()}

In [17]:
# Display the mappings
print("labels_to_ids:", labels_to_ids_test)
print("ids_to_labels:", ids_to_labels_test)

labels_to_ids: {'B-ORG': 0, 'B-CHAR': 1, 'I-CHAR': 2, 'I-LOC': 3, 'B-LOC': 4, 'O': 5}
ids_to_labels: {0: 'B-ORG', 1: 'B-CHAR', 2: 'I-CHAR', 3: 'I-LOC', 4: 'B-LOC', 5: 'O'}


In [18]:
training_set = dataset(df, tokenizer, MAX_LEN)

In [19]:
testing_set = dataset(df_test, tokenizer, MAX_LEN)

In [20]:
# testing_set[0]

In [21]:
# for token, label in zip(tokenizer.convert_ids_to_tokens(testing_set[0]["input_ids"]), testing_set[0]["labels"]):
#   print('{0:10}  {1}'.format(token, label))

In [22]:
# for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
#   print('{0:10}  {1}'.format(token, label))

In [23]:
# def decode_tokens_and_labels(tokenizer, encoded_input, ids_to_labels):
#     # Decode the input_ids to tokens
#     tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'].tolist())
    
#     # Convert label IDs back to label names
#     label_ids = encoded_input['labels'].tolist()
#     labels = [ids_to_labels[label_id] if label_id != -100 else 'PAD' for label_id in label_ids]
    
#     # Combine tokens and labels
#     decoded = [(token, label) for token, label in zip(tokens, labels)]
    
#     return decoded

# # Example usage:
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# encoded_input = training_set[0]
# decoded_output = decode_tokens_and_labels(tokenizer, encoded_input, ids_to_labels)

# for token, label in decoded_output:
#     print(f"{token}: {label}")

In [24]:
# import numpy as np
# import pandas as pd
# import torch
# from torch.utils.data import Dataset
# from transformers import BertTokenizerFast

# class CustomDataset(Dataset):
#     def __init__(self, dataframe, tokenizer, max_len):
#         self.len = len(dataframe)
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __getitem__(self, index):
#         # step 1: get the sentence and word labels
#         sentence = self.data.sentence[index].strip().split()
#         word_labels = self.data.word_labels[index].split(",")

#         # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
#         # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
#         encoding = self.tokenizer(sentence,
#                                   is_split_into_words=True,
#                                   return_offsets_mapping=True,
#                                   padding='max_length',
#                                   truncation=True,
#                                   max_length=self.max_len)

#         # step 3: create token labels, handling subwords correctly
#         labels = [labels_to_ids[label] for label in word_labels]
#         encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

#         # set labels for subwords
#         i = 0
#         for idx, mapping in enumerate(encoding["offset_mapping"]):
#             if mapping[0] == 0 and mapping[1] != 0:
#                 encoded_labels[idx] = labels[i]
#                 i += 1
#             elif mapping[0] != 0 and mapping[1] != 0:
#                 encoded_labels[idx] = labels[i-1] if word_labels[i-1].startswith("B-") else labels[i-1]

#         # step 4: turn everything into PyTorch tensors
#         item = {key: torch.as_tensor(val) for key, val in encoding.items()}
#         item['labels'] = torch.as_tensor(encoded_labels)

#         return item

#     def __len__(self):
#         return self.len

# def read_data(file_path):
#     sentences, labels = [], []
#     sentence, label = [], []
#     with open(file_path, encoding="utf-8") as file:
#         for line in file:
#             if line.startswith("#"):
#                 continue
#             elif line == "\n":
#                 if sentence:
#                     sentences.append(sentence)
#                     labels.append(label)
#                     sentence, label = [], []
#             else:
#                 parts = line.strip().split("\t")
#                 sentence.append(parts[1].lower())  # Convert the token to lowercase before appending
#                 label.append(clean_tag(parts[2]))
#     if sentence:
#         sentences.append(sentence)
#         labels.append(label)
#     return sentences, labels

# def clean_tag(tag):
#     # Ensure tags are in the correct format
#     if tag.count('-') > 1:
#         prefix, entity = tag.split('-', 1)
#         tag = f"{prefix}-{entity.replace('-', '')}"
#     return tag

# # Example usage
# train_tokens, train_tags = read_data("./tagged_sentences_train.iob2")
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# data = {'sentence': [" ".join(sentence) for sentence in train_tokens],
#         'word_labels': [",".join(tags) for tags in train_tags]}

# df = pd.DataFrame(data)

# # Split the word_labels into individual tags and get unique tags
# all_tags = [tag for tags in df['word_labels'] for tag in tags.split(",")]
# unique_tags = set(all_tags)
# print(unique_tags)
# print()
# # Create mappings
# labels_to_ids = {k: v for v, k in enumerate(unique_tags)}
# ids_to_labels = {v: k for k, v in labels_to_ids.items()}

# MAX_LEN = 128
# training_set = CustomDataset(df, tokenizer, MAX_LEN)
# decoded_output = decode_tokens_and_labels(tokenizer, training_set[1], ids_to_labels)

# for token, label in decoded_output:
#     print(f"{token}: {label}")


In [25]:
train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [26]:
len(labels_to_ids)

6

In [27]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [28]:
import math
print(-math.log(1/(len(labels_to_ids))))

1.791759469228055


In [29]:
inputs = training_set[7]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(1.6472, device='cuda:0', grad_fn=<NllLossBackward0>)

In [30]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 174, 6])

In [31]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-5)

In [32]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds,tr_labels = [], []

    # Put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        # Forward pass
        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs.loss
        tr_logits = outputs.logits

        # Accumulate the training loss
        tr_loss += loss.item()
        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # Compute training accuracy
        flattened_targets = labels.view(-1)  # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.config.num_labels)  # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

        # Only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100  # shape (batch_size * seq_len,)
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels.cpu().numpy())
        tr_preds.extend(predictions.cpu().numpy())

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)


Training epoch: 1
Training loss per 100 training steps: 1.7520294189453125
Training loss per 100 training steps: 0.12566136439690495
Training loss per 100 training steps: 0.0784658856866001
Training loss per 100 training steps: 0.05856132630469197
Training loss epoch: 0.056738034113173845
Training accuracy epoch: 0.9874865240657869
Training epoch: 2
Training loss per 100 training steps: 0.00999098364263773
Training loss per 100 training steps: 0.010535700850668225
Training loss per 100 training steps: 0.009953313412733228
Training loss per 100 training steps: 0.00957212043443441
Training loss epoch: 0.009429574792813153
Training accuracy epoch: 0.9973247840704456
Training epoch: 3
Training loss per 100 training steps: 0.0031392767559736967
Training loss per 100 training steps: 0.005265654144822482
Training loss per 100 training steps: 0.005301643163834546
Training loss per 100 training steps: 0.0050477860340743365
Training loss epoch: 0.005081990875739747
Training accuracy epoch: 0.998

KeyboardInterrupt: 

In [ ]:
# # Define the path to the saved model directory
# directory = "./model"

# # Load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(directory)

# # Load the model
# model = BertForTokenClassification.from_pretrained(directory)
# model.to(device)

# print('Model and tokenizer loaded successfully')


In [33]:
model.save_pretrained("bert_train")

In [34]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [35]:
import json
config = json.load(open('bert_train/config.json'))
config['id2label'] = ids_to_labels
config['label2id'] = labels_to_ids
json.dump(config, open('bert_train/config.json', 'w'))
model = BertForTokenClassification.from_pretrained('bert_train', num_labels=len(labels_to_ids))
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [36]:

model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
eval_preds, eval_labels = [], []

with torch.no_grad():
    for idx, batch in enumerate(testing_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs.loss
        eval_logits = outputs.logits

        eval_loss += loss.item()
        nb_eval_steps += 1
        nb_eval_examples += labels.size(0)

        flattened_targets = labels.view(-1)
        active_logits = eval_logits.view(-1, model.config.num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)

        active_accuracy = labels.view(-1) != -100
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        eval_labels.extend(labels.cpu().numpy())
        eval_preds.extend(predictions.cpu().numpy())

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = accuracy_score(eval_labels, eval_preds)

print(f"Evaluation loss: {eval_loss}")
print(f"Evaluation accuracy: {eval_accuracy}")

report = classification_report(eval_labels, eval_preds, target_names=list(labels_to_ids.keys()))
print("Classification Report:")
print(report)

Evaluation loss: 0.05485333547823959
Evaluation accuracy: 0.9900095693779905
Classification Report:
              precision    recall  f1-score   support

       B-ORG       0.00      0.00      0.00         2
      B-CHAR       0.86      0.86      0.86       820
      I-CHAR       0.89      0.78      0.83        85
       I-LOC       0.00      0.00      0.00         2
       B-LOC       0.94      0.97      0.95       216
           O       0.99      1.00      1.00     25000

    accuracy                           0.99     26125
   macro avg       0.61      0.60      0.61     26125
weighted avg       0.99      0.99      0.99     26125



/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [37]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)

            # Forward pass
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs.loss
            eval_logits = outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # Compute evaluation accuracy
            flattened_targets = labels.view(-1)  # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.config.num_labels)  # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

            # Only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100  # shape (batch_size * seq_len,)
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels.cpu().numpy())
            eval_preds.extend(predictions.cpu().numpy())

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id] for id in eval_labels]
    predictions = [ids_to_labels[id] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions



In [38]:
# Evaluation function
def evaluate():
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs.loss
            eval_logits = outputs.logits

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            flattened_targets = labels.view(-1)
            active_logits = eval_logits.view(-1, model.config.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)

            active_accuracy = labels.view(-1) != -100
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels.cpu().numpy())
            eval_preds.extend(predictions.cpu().numpy())

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = accuracy_score(eval_labels, eval_preds)

    print(f"Evaluation loss: {eval_loss}")
    print(f"Evaluation accuracy: {eval_accuracy}")

    # Get sorted list of label IDs
    sorted_labels = sorted(labels_to_ids, key=labels_to_ids.get)

    report = classification_report(eval_labels, eval_preds, labels=[labels_to_ids[label] for label in sorted_labels], target_names=sorted_labels)
    print("Classification Report:")
    print(report)
# evaluate()

In [39]:
labels, predictions = valid(model, testing_loader)


Validation loss per 100 evaluation steps: 0.05313334986567497
Validation Loss: 0.05507717819677459
Validation Accuracy: 0.9898758699840837


In [40]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)

            # Forward pass
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs.loss
            eval_logits = outputs.logits

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # Compute evaluation accuracy
            active_logits = eval_logits.view(-1, model.config.num_labels)  # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)  # shape (batch_size * seq_len,)

            for i in range(labels.size(0)):
                label = labels[i]
                pred = flattened_predictions.view(labels.size(0), labels.size(1))[i]

                active_accuracy = label != -100  # shape (seq_len,)
                label = torch.masked_select(label, active_accuracy)
                pred = torch.masked_select(pred, active_accuracy)

                eval_labels.append([ids_to_labels[id.item()] for id in label])
                eval_preds.append([ids_to_labels[id.item()] for id in pred])

                tmp_eval_accuracy = accuracy_score(label.cpu().numpy(), pred.cpu().numpy())
                eval_accuracy += tmp_eval_accuracy

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds

In [41]:
# Example usage
labels, predictions = valid(model, testing_loader)


Validation loss per 100 evaluation steps: 0.037515584379434586
Validation Loss: 0.05498418704503112
Validation Accuracy: 62.92922030032321


In [42]:
from seqeval.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

        CHAR       0.84      0.84      0.84       820
         LOC       0.93      0.96      0.95       216
         ORG       0.00      0.00      0.00         2

   micro avg       0.86      0.87      0.86      1038
   macro avg       0.59      0.60      0.60      1038
weighted avg       0.86      0.87      0.86      1038



/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
tokenizer = BertTokenizerFast.from_pretrained("tokenizer")


In [50]:
sentence = "Oooohhh Frodo Aaaahh Gimli My precious Wake up Wake up Wake up sleepies We must go yes we must go at once"
words = sentence.split()
inputs = tokenizer(words,
             is_split_into_words=True,
             return_offsets_mapping=True,
             padding='max_length',
             truncation=True,
             max_length=MAX_LEN,
             return_tensors="pt")


# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
  #only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue

print(sentence.split())
print(prediction)

['Oooohhh', 'Frodo', 'Aaaahh', 'Gimli', 'My', 'precious', 'Wake', 'up', 'Wake', 'up', 'Wake', 'up', 'sleepies', 'We', 'must', 'go', 'yes', 'we', 'must', 'go', 'at', 'once']
['O', 'B-CHAR', 'O', 'B-CHAR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [51]:
import torch

# Ensure MAX_LEN is defined
MAX_LEN = 128

sentence = "Oh Smeagol Ive got one Ive got a fish Smeagol Smeagol Pull it in Go on go on go on pull it in Arrghh Deagol Deagol Deagol Give us that Deagol my love Why"
words = sentence.split()
inputs = tokenizer(words,
             is_split_into_words=True,
             return_offsets_mapping=True,
             padding='max_length',
             truncation=True,
             max_length=MAX_LEN,
             return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs.logits

active_logits = logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
  # only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue

print(words)
print(prediction)


['Oh', 'Smeagol', 'Ive', 'got', 'one', 'Ive', 'got', 'a', 'fish', 'Smeagol', 'Smeagol', 'Pull', 'it', 'in', 'Go', 'on', 'go', 'on', 'go', 'on', 'pull', 'it', 'in', 'Arrghh', 'Deagol', 'Deagol', 'Deagol', 'Give', 'us', 'that', 'Deagol', 'my', 'love', 'Why']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [57]:
from transformers import pipeline

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1510, in _get_module
    def __dir__(self):
  File "/home/cogu/.conda/envs/bert-try/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/cogu/.conda/envs/bert-try/lib/python3.10/site-packages/transformers/pipelines/__init__.py", line 47, in <module>
    from .audio_classification import AudioClassificationPipeline
  File "/home

In [45]:
# import os

# directory = "./model"

# if not os.path.exists(directory):
#     os.makedirs(directory)

# # save vocabulary of the tokenizer
# tokenizer.save_vocabulary(directory)
# # save the model weights and its configuration file
# model.save_pretrained(directory)
# print('All files saved')


In [46]:

# Display the mappings
print("labels_to_ids:", labels_to_ids)
print("ids_to_labels:", ids_to_labels)

labels_to_ids: {'B-ORG': 0, 'B-CHAR': 1, 'I-CHAR': 2, 'I-LOC': 3, 'B-LOC': 4, 'O': 5}
ids_to_labels: {0: 'B-ORG', 1: 'B-CHAR', 2: 'I-CHAR', 3: 'I-LOC', 4: 'B-LOC', 5: 'O'}


In [47]:
import json
config = json.load(open('bert_train/config.json'))
config['id2label'] = ids_to_labels
config['label2id'] = labels_to_ids
json.dump(config, open('bert_train/config.json', 'w'))
model = BertForTokenClassification.from_pretrained('bert_train', num_labels=len(labels_to_ids))
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el